In [32]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU,SimpleRNN
from tensorflow.keras.models import Model
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from transformers import TFBertModel,  BertConfig, BertTokenizerFast


In [18]:
# Detect hardware, return appropriate distribution strategy
try:
    # TPU detection. No parameters necessary if TPU_NAME environment variable is
    # set: this is always the case on Kaggle.
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    # Default distribution strategy in Tensorflow. Works on CPU and single GPU.
    strategy = tf.distribute.get_strategy()

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [19]:
model_name = 'bert-base-multilingual-cased'

config = BertConfig.from_pretrained(model_name)
config.output_hidden_states = False
tokenizer = BertTokenizerFast.from_pretrained(pretrained_model_name_or_path = model_name, config = config)
transformer_model = TFBertModel.from_pretrained(model_name, config = config)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the model checkpoint at bert-base-multilingual-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [20]:
train = pd.read_csv('../dataset/train.csv')
valid = pd.read_csv('../dataset/val.csv')
test = pd.read_csv('../dataset/test.csv')

In [21]:
train

,Emotion,Comment
0,0,On days when I feel close to my partner and ot...
1,1,Every time I imagine that someone I love or I ...
2,2,When I had been obviously unjustly treated and...
3,3,When I think about the short time that we live...
4,4,At a gathering I found myself involuntarily si...
...,...,...
5179,1,My parents were out and I was the eldest at ho...
5180,5,Two years back someone invited me to be the tu...
5181,5,I had taken the responsibility to do something...
5182,1,I was at home and I heard a loud sound of spit...


In [22]:
#IMP DATA FOR CONFIG

AUTO = tf.data.experimental.AUTOTUNE


# Configuration
EPOCHS = 3
BATCH_SIZE = 16 * strategy.num_replicas_in_sync
MAX_LEN = 168

In [23]:
xtrain, xvalid, ytrain, yvalid = train_test_split(train.Comment.values, train.Emotion.values, 
                                                  stratify=train.Emotion.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

train['Comment'].apply(lambda x:len(str(x).split())).max()

168

In [24]:
yvalid.shape

(1037,)

In [25]:
# using keras tokenizer here
token = text.Tokenizer(num_words=None)
max_len = 168

token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

#zero pad the sequences
xtrain_pad = sequence.pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = sequence.pad_sequences(xvalid_seq, maxlen=max_len)

word_index = token.word_index


In [26]:
xvalid_pad.shape

(1037, 168)

In [27]:
def build_model(transformer):
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    sequence_output = transformer(input_ids)[0]
    cls_token = sequence_output[:, 0, :]
    out = Dense(7, activation='softmax')(cls_token)

    model = Model(inputs=input_ids, outputs=out)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    model.compile(Adam(lr=1e-5), loss=loss, metrics=['accuracy'])

    return model

In [28]:
metrics = [tf.keras.metrics.SparseCategoricalAccuracy('accuracy', dtype=tf.float32)]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

In [33]:
with strategy.scope():
    # A simpleRNN without any pretrained embeddings and one dense layer
    # model = Sequential()
    # model.add(Embedding(len(word_index) + 1,300,input_length=max_len))
    # model.add(SimpleRNN(100))
    # model.add(Dense(7, activation='softmax'))
    # model.compile(loss=loss, optimizer='adam', metrics=['accuracy'])
    model = build_model(transformer_model)
    
model.summary()

NameError: name 'Adam' is not defined

In [15]:
model.fit(xtrain_pad, ytrain,epochs=10, batch_size=64*strategy.num_replicas_in_sync) #Multiplying by Strategy to run on TPU's

Epoch 1/10
65/65 [==============================] - 6s 90ms/step - loss: 1.9378 - accuracy: 0.1736
Epoch 2/10
65/65 [==============================] - 6s 94ms/step - loss: 1.7832 - accuracy: 0.5293
Epoch 3/10
65/65 [==============================] - 6s 98ms/step - loss: 1.4435 - accuracy: 0.8257
Epoch 4/10
65/65 [==============================] - 7s 101ms/step - loss: 1.2794 - accuracy: 0.9289
Epoch 5/10
65/65 [==============================] - 7s 103ms/step - loss: 1.2158 - accuracy: 0.9674
Epoch 6/10
65/65 [==============================] - 7s 106ms/step - loss: 1.1925 - accuracy: 0.9826
Epoch 7/10
65/65 [==============================] - 7s 104ms/step - loss: 1.1824 - accuracy: 0.9879
Epoch 8/10
65/65 [==============================] - 7s 107ms/step - loss: 1.1776 - accuracy: 0.9913
Epoch 9/10
65/65 [==============================] - 7s 102ms/step - loss: 1.1746 - accuracy: 0.9932
Epoch 10/10
65/65 [==============================] - 7s 101ms/step - loss: 1.1731 - accuracy: 0.9940


In [16]:
model.evaluate(xvalid_pad, yvalid)

33/33 [==============================] - 1s 18ms/step - loss: 1.8483 - accuracy: 0.2932


[1.8483151197433472, 0.293153315782547]